<a href="https://colab.research.google.com/github/reachosen/No_Surprises_Act_idr/blob/main/NoSurprisesActComplianceIDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the NSA IDR context:

True Positive (TP): The system correctly identifies a claim where the billed amount exceeds the QPA but is justified (e.g., due to emergency complexity or expertise), providing supporting insights.
True Negative (TN): The system correctly identifies a claim where the billed amount aligns with or is below the QPA/median rate, suggesting no adjustment is needed.
False Positive (FP): The system incorrectly suggests a higher payment is justified when the billed amount is excessive and lacks supporting factors (e.g., no complexity or expertise).
False Negative (FN): The system fails to recognize a justified higher payment, suggesting alignment with QPA despite clear complexity or other factors.

In [ ]:
test_cases = {
    "TP_emergency_high_bill": {
        "description": "Out-of-network ER visit, CPT 99283, billed $1,500 due to emergency complexity and patient stabilization.",
        "billed_amount": 1500.0,
        "qpa": 1000.0,
        "provider_experience": 12,
        "is_emergency": True,
        "expected_insight": "Justified higher payment due to emergency context and expertise."
    },
    "TN_routine_low_bill": {
        "description": "Out-of-network office visit, CPT 99213, billed $900, routine care.",
        "billed_amount": 900.0,
        "qpa": 950.0,
        "provider_experience": 3,
        "is_emergency": False,
        "expected_insight": "Billed amount aligns with QPA; no adjustment needed."
    },
    "FP_excessive_no_justification": {
        "description": "Out-of-network consult, CPT 99204, billed $2,000, standard procedure.",
        "billed_amount": 2000.0,
        "qpa": 1200.0,
        "provider_experience": 2,
        "is_emergency": False,
        "expected_insight": "Excessive billing without complexity; adjustment unlikely."
    },
    "FN_missed_complexity": {
        "description": "Out-of-network surgery, CPT 27447, billed $10,000, complex knee replacement in rural area.",
        "billed_amount": 10000.0,
        "qpa": 8000.0,
        "provider_experience": 15,
        "is_emergency": False,
        "expected_insight": "Complex procedure and expertise may justify higher payment."
    }
}

In [ ]:
# Set your Groq API key
GROQ_API_KEY = ""  # Replace with your actual API key

In [ ]:
import json
import requests
import re
from dataclasses import dataclass
from typing import Dict

# Data structures
@dataclass
class Claim:
    description: str
    billed_amount: float
    qpa: float
    provider_experience: int
    is_emergency: bool

@dataclass
class SupportingData:
    median_rate: float
    historical_outcomes: list
    external_trends: str

# Groq API Configuration

GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"

# Generative AI with Groq
class GroqAI:
    def analyze_claim(self, claim: Claim, supporting: SupportingData) -> str:
        system_message = (
            "You are an expert in healthcare billing disputes under the No Surprises Act. "
            "Analyze the claim data, compare it to supporting data, and generate an impartial summary for IDR. "
            "Do not determine final payment amounts; focus on insights."
        )
        user_prompt = (
            f"Claim: {claim.description}\n"
            f"Billed Amount: ${claim.billed_amount}, QPA: ${claim.qpa}\n"
            f"Provider Experience: {claim.provider_experience} years\n"
            f"Emergency: {'Yes' if claim.is_emergency else 'No'}\n"
            f"Supporting Data:\n- Median Rate: ${supporting.median_rate}\n"
            f"- Historical Outcomes: {', '.join([f'${x}' for x in supporting.historical_outcomes])}\n"
            f"- External Trends: {supporting.external_trends}\n"
            "Provide a detailed analysis for IDR consideration."
        )
        payload = {
            "model": "llama3-70b-8192",
            "messages": [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_prompt}
            ],
            "temperature": 0.5,
            "max_tokens": 1024,
            "top_p": 1.0,
            "stream": False
        }
        headers = {
            "Authorization": f"Bearer {GROQ_API_KEY}",
            "Content-Type": "application/json"
        }
        print(f"Calling Groq API for {claim.description[:20]}...")
        response = requests.post(GROQ_URL, headers=headers, json=payload)
        if response.status_code == 200:
            return response.json()["choices"][0]["message"]["content"]
        else:
            raise ValueError(f"API failed: {response.status_code} - {response.text}")

# IDR System
class IDRSupportSystem:
    def __init__(self):
        self.ai = GroqAI()
        self.supporting_data_db = {
            "CPT 99283": SupportingData(median_rate=1050.0, historical_outcomes=[1100.0, 1200.0, 1000.0], external_trends="60% settle $1,100-$1,300"),
            "CPT 99213": SupportingData(median_rate=925.0, historical_outcomes=[900.0, 950.0, 920.0], external_trends="Routine visits settle near QPA"),
            "CPT 99204": SupportingData(median_rate=1250.0, historical_outcomes=[1200.0, 1300.0, 1150.0], external_trends="Standard consults rarely exceed $1,400"),
            "CPT 27447": SupportingData(median_rate=8200.0, historical_outcomes=[8500.0, 9000.0, 8000.0], external_trends="Complex surgeries in rural areas settle 10-20% above QPA")
        }

    def extract_cpt_code(self, description: str) -> str:
        # Use regex to extract CPT code (e.g., "CPT 99283" or "99283")
        match = re.search(r"CPT\s*(\d{5})|\b(\d{5})\b", description)
        if match:
            cpt = match.group(1) or match.group(2)
            return f"CPT {cpt}"
        return None

    def process_claim(self, claim_data: Dict) -> str:
        claim = Claim(**{k: v for k, v in claim_data.items() if k != "expected_insight"})
        cpt_code = self.extract_cpt_code(claim.description)
        if not cpt_code or cpt_code not in self.supporting_data_db:
            return f"Error: No supporting data for {cpt_code or 'unknown CPT'}"
        supporting = self.supporting_data_db[cpt_code]
        return self.ai.analyze_claim(claim, supporting)

    def generate_report(self, claim_data: Dict) -> str:
        analysis = self.process_claim(claim_data)
        report = [
            "IDR Support Report",
            "Generated on: March 03, 2025",
            "----------------------------------------",
            f"Expected Insight: {claim_data['expected_insight']}",
            "Actual Analysis:",
            analysis,
            "----------------------------------------",
            "Note: AI-generated analysis for IDR support."
        ]
        return "\n".join(report)

# Test Cases
test_cases = {
    "TP_emergency_high_bill": {
        "description": "Out-of-network ER visit, CPT 99283, billed $1,500 due to emergency complexity and patient stabilization.",
        "billed_amount": 1500.0,
        "qpa": 1000.0,
        "provider_experience": 12,
        "is_emergency": True,
        "expected_insight": "Justified higher payment due to emergency context and expertise."
    },
    "TN_routine_low_bill": {
        "description": "Out-of-network office visit, CPT 99213, billed $900, routine care.",
        "billed_amount": 900.0,
        "qpa": 950.0,
        "provider_experience": 3,
        "is_emergency": False,
        "expected_insight": "Billed amount aligns with QPA; no adjustment needed."
    },
    "FP_excessive_no_justification": {
        "description": "Out-of-network consult, CPT 99204, billed $2,000, standard procedure.",
        "billed_amount": 2000.0,
        "qpa": 1200.0,
        "provider_experience": 2,
        "is_emergency": False,
        "expected_insight": "Excessive billing without complexity; adjustment unlikely."
    },
    "FN_missed_complexity": {
        "description": "Out-of-network surgery, CPT 27447, billed $10,000, complex knee replacement in rural area.",
        "billed_amount": 10000.0,
        "qpa": 8000.0,
        "provider_experience": 15,
        "is_emergency": False,
        "expected_insight": "Complex procedure and expertise may justify higher payment."
    }
}

# Run Tests
def main():
    idr_system = IDRSupportSystem()
    for case_name, claim_data in test_cases.items():
        print(f"\nTesting {case_name}:")
        report = idr_system.generate_report(claim_data)
        print(report)

if __name__ == "__main__":
    main()


Testing TP_emergency_high_bill:
Calling Groq API for Out-of-network ER vi...
IDR Support Report
Generated on: March 03, 2025
----------------------------------------
Expected Insight: Justified higher payment due to emergency context and expertise.
Actual Analysis:
**Impartial Summary for IDR Consideration**

**Claim Overview**

The claim in dispute is for an out-of-network emergency room (ER) visit, coded as CPT 99283, with a billed amount of $1,500. The provider, with 12 years of experience, argues that the complexity of the emergency and patient stabilization justify the higher charge.

**Key Findings**

1. **Qualified Payment Amount (QPA)**: The QPA is $1,000, which serves as a baseline for the IDR process.
2. **Median Rate**: The median rate for similar services in the area is $1,050, which is higher than the QPA but lower than the billed amount.
3. **Historical Outcomes**: Historical outcomes for similar claims suggest a range of $1,000 to $1,200, which is closer to the QPA and 